<a href="https://colab.research.google.com/github/Nargiz-chess/Solar_forecasting/blob/main/ConvLSTM_AUSTRALIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Mount google drive
from google.colab import drive
drive.mount('/content/drive')

from pandas import read_csv
import pandas as pd
import datetime as dt
import matplotlib
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

# multivariate multi-step encoder-decoder lstm
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D



Mounted at /content/drive


In [2]:
path1 = '/content/drive/MyDrive/Master_thesis/Datasets/Australia_numpy_removed_night_hours.npy'
path2 = '/content/drive/MyDrive/Master_thesis/Datasets/irradiance_AUS(2012-2013).csv'

In [ ]:
array = np.loadtxt(path1)
print("Shape before reshape",np.shape(array))
AUS_data = array.reshape(array.shape[0]*array.shape[1], order="F")
print("Shape after reshape",np.shape(AUS_data))

Shape before reshape (6570, 299)
Shape after reshape (1964430,)


In [ ]:
df2=read_csv(path2, header =0, index_col=0) 
df2.shape
df2=df2.drop("time", axis=1)
array2 = df2.values
weath_data = array2.reshape(array2.shape[0]*array2.shape[1])
print("Shape of Aus data = ", AUS_data.shape, "\n","Shape of weather data = ", weath_data.shape )


Shape of Aus data =  (1964430,) 
 Shape of weather data =  (6570,)


In [ ]:
#stacking irradiance as many times as there are houses in the dataset to match with each house in the datset

# weath_data_stacked = np.tile(weath_data,299)
# weath_data_stacked.shape
# data= np.column_stack((AUS_data,weath_data_stacked))
# data.shape
# data = np.nan_to_num(data)
# data = pd.DataFrame(data=data)
data.shape


(1964430, 2)

In [ ]:
from numpy import split
from numpy import array


# split a univariate dataset into train/test sets
def split_dataset(data):
  # split into standard weeks
  train, test = data[:-90], data[-90:]
  # restructure into windows of weekly data
  train = array(split(train, len(train)/18))
  test = array(split(test, len(test)/18))
  return train, test

# evaluate one or more weekly forecasts against expected values
def evaluate_forecasts(actual, predicted):
  scores = list()
  # calculate an RMSE score for each day
  for i in range(actual.shape[1]):
    # calculate mse
    mse = mean_squared_error(actual[:, i], predicted[:, i])
    # calculate rmse
    rmse = sqrt(mse)
    # store
    scores.append(rmse)
  # calculate overall RMSE
  s = 0
  for row in range(actual.shape[0]):
    for col in range(actual.shape[1]):
      s += (actual[row, col] - predicted[row, col])**2
  score = sqrt(s / (actual.shape[0] * actual.shape[1]))
  return score, scores

# summarize scores
def summarize_scores(name, score, scores):
  s_scores = ', '.join(['%.1f' % s for s in scores])
  print('%s: [%.3f] %s' % (name, score, s_scores))
  
# convert history into inputs and outputs
def to_supervised(train, n_input, n_out=18):
  # flatten data
  data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
  X, y = list(), list()
  in_start = 0
  # step over the entire history one time step at a time
  for _ in range(len(data)):
    # define the end of the input sequence
    in_end = in_start + n_input
    out_end = in_end + n_out
    # ensure we have enough data for this instance
    # Here we are dividing the data differently because we are working with a multivariate dataset (It contains more variables) 
    if out_end < len(data):
      X.append(data[in_start:in_end, :])
      y.append(data[in_end:out_end, 0])
    # move along one time step
    in_start += 1
  return array(X), array(y)

# train the model
def build_model(train, n_steps, n_length, n_input):
  # prepare data
  train_x, train_y = to_supervised(train, n_input)
  # define parameters
  verbose, epochs, batch_size = 1, 1, 54
  n_features, n_outputs = train_x.shape[2], train_y.shape[1]
  # reshape into subsequences [samples, timesteps, rows, cols, channels]
  train_x = train_x.reshape((train_x.shape[0], n_steps, 1, n_length, n_features))
  # reshape output into [samples, timesteps, features]
  train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))
  # define model
  model = Sequential()
  model.add(ConvLSTM2D(filters=20, kernel_size=(1,3), activation='relu',input_shape=(n_steps, 1, n_length, n_features)))
  model.add(Flatten())
  model.add(RepeatVector(n_outputs))
  model.add(LSTM(20, activation='relu', return_sequences=True))
  #model.add(LSTM(20, activation='relu', return_sequences=True))
  model.add(TimeDistributed(Dense(100, activation='relu')))
  model.add(TimeDistributed(Dense(1)))
  model.compile(loss='mse', optimizer='adam')
  # fit network
  model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose)
  return model

# make a forecast
def forecast(model, history, n_steps, n_length, n_input):
  # flatten data
  data = array(history)
  data = data.reshape((data.shape[0]*data.shape[1], data.shape[2]))
  # retrieve last observations for input data
  input_x = data[-n_input:, :]
  # reshape into [samples, timesteps, rows, cols, channels]
  input_x = input_x.reshape((1, n_steps, 1, n_length, 2))
  # forecast the next week
  yhat = model.predict(input_x, verbose=0)
  # we only want the vector forecast
  yhat = yhat[0]
  return yhat

# evaluate a single model
def evaluate_model(train, test, n_steps, n_length, n_input):
  # fit model
  model = build_model(train, n_steps, n_length, n_input)
  # history is a list of weekly data
  history = [x for x in train]
  # walk-forward validation over each week
  predictions = list()
  for i in range(len(test)):
    # predict the next day
    yhat_sequence = forecast(model, history, n_steps, n_length, n_input)
    # store the predictions
    predictions.append(yhat_sequence)
    # get real observation and add to history for predicting the next week
    history.append(test[i, :])
  # evaluate predictions days for each week
  predictions = array(predictions)
  score, scores = evaluate_forecasts(test[:, :, 0], predictions)
  return score, scores,predictions


# # split into train and test
# train, test = split_dataset(data.values)
# # define the number of subsequences and the length of subsequences
# n_steps, n_length = 5, 18
# # define the total days to use as input
# n_input = n_length * n_steps
# score, scores,predictions = evaluate_model(train, test, n_steps, n_length, n_input)
# # summarize scores
# summarize_scores('lstm', score, scores)
# # # plot scores

# #hours = [x for x in range(1,19)]
# hours = [str(x) for x in range(1,19)]
# pyplot.plot(hours, scores, marker='o', label='lstm')
# pyplot.show()


In [ ]:
def cross_val (array, samples_division):
  array = np.concatenate((array[-samples_division:],array[:-samples_division] ))
  return array

scores_cv = []
for i in range(1):
  data = cross_val(data,6570)
  # split into train and test
  train, test = split_dataset(data)
  # define the number of subsequences and the length of subsequences
  n_steps, n_length = 5, 18
  # define the total days to use as input
  n_input = n_length * n_steps
  score, scores,predictions = evaluate_model(train, test, n_steps, n_length, n_input)
  # summarize scores
  summarize_scores('lstm', score, scores)
  scores_cv.append(score)
  print(data[0])


36375/36375 [==============================] - 1741s 48ms/step - loss: 2.5366
lstm: [0.287] 0.3, 0.3, 0.3, 0.3, 0.3, 0.2, 0.1, 0.1, 0.2, 0.2, 0.2, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3
[0. 0.]
36375/36375 [==============================] - 1706s 47ms/step - loss: 1.9398
lstm: [0.061] 0.0, 0.0, 0.0, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
[0. 0.]


In [ ]:
import random

a = np.random.rand(3,2)
print(a)

[[0.33296593 0.90165975]
 [0.80686784 0.8513707 ]
 [0.05656842 0.04594226]]


# SUBSET: random 100 houses

In [3]:
import random

random.seed(42)
subset_col = [random.randint(0, 298) for p in range(0, 100)]
print(subset_col)

[57, 12, 140, 125, 114, 71, 52, 279, 44, 216, 16, 15, 47, 111, 119, 258, 13, 287, 101, 279, 214, 112, 229, 142, 3, 81, 216, 174, 142, 79, 110, 172, 52, 47, 194, 49, 183, 176, 135, 22, 235, 274, 63, 193, 40, 282, 150, 185, 295, 98, 35, 23, 116, 148, 40, 119, 51, 194, 142, 232, 186, 83, 189, 181, 107, 136, 36, 87, 273, 125, 83, 236, 194, 138, 285, 112, 166, 28, 117, 16, 161, 205, 137, 33, 108, 290, 161, 108, 255, 202, 234, 73, 135, 71, 126, 287, 275, 134, 219, 298]


In [4]:
array = np.loadtxt(path1)
print("Shape before reshape",np.shape(array))

array = array[:,subset_col]
print(array.shape)

Shape before reshape (6570, 299)
(6570, 100)


In [5]:
AUS_data = array.reshape(array.shape[0]*array.shape[1], order="F")
print("Shape after reshape",np.shape(AUS_data))

Shape after reshape (657000,)


In [6]:
df2=read_csv(path2, header =0, index_col=0) 
df2.shape
df2=df2.drop("time", axis=1)
array2 = df2.values
weath_data = array2.reshape(array2.shape[0]*array2.shape[1])
print("Shape of Aus data = ", AUS_data.shape, "\n","Shape of weather data = ", weath_data.shape )


Shape of Aus data =  (657000,) 
 Shape of weather data =  (6570,)


In [7]:
#stacking irradiance as many times as there are houses in the dataset to match with each house in the datset

weath_data_stacked = np.tile(weath_data,100)
weath_data_stacked.shape
data= np.column_stack((AUS_data,weath_data_stacked))
data.shape
data = np.nan_to_num(data)
data = pd.DataFrame(data=data)
data.shape

(657000, 2)

In [41]:
from numpy import split
from numpy import array


# split a univariate dataset into train/test sets
def split_dataset(data):
  # split into standard weeks
  train, test = data[:-90], data[-90:]
  # restructure into windows of weekly data
  train = array(split(train, len(train)/18))
  test = array(split(test, len(test)/18))
  return train, test

# evaluate one or more weekly forecasts against expected values
def evaluate_forecasts(actual, predicted):
  scores = list()
  # calculate an RMSE score for each day
  for i in range(actual.shape[1]):
    # calculate mse
    mse = mean_squared_error(actual[:, i], predicted[:, i])
    # calculate rmse
    rmse = sqrt(mse)
    # store
    scores.append(rmse)
  # calculate overall RMSE
  s = 0
  for row in range(actual.shape[0]):
    for col in range(actual.shape[1]):
      s += (actual[row, col] - predicted[row, col])**2
  score = sqrt(s / (actual.shape[0] * actual.shape[1]))
  return score, scores

# summarize scores
def summarize_scores(name, score, scores):
  s_scores = ', '.join(['%.1f' % s for s in scores])
  print('%s: [%.3f] %s' % (name, score, s_scores))
  
# convert history into inputs and outputs
def to_supervised(train, n_input, n_out=18):
  # flatten data
  data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
  X, y = list(), list()
  in_start = 0
  # step over the entire history one time step at a time
  for _ in range(len(data)):
    # define the end of the input sequence
    in_end = in_start + n_input
    out_end = in_end + n_out
    # ensure we have enough data for this instance
    # Here we are dividing the data differently because we are working with a multivariate dataset (It contains more variables) 
    if out_end < len(data):
      X.append(data[in_start:in_end, :])
      y.append(data[in_end:out_end, 0])
    # move along one time step
    in_start += 1
  return array(X), array(y)

# train the model
def build_model(train, n_steps, n_length, n_input):
  # prepare data
  train_x, train_y = to_supervised(train, n_input)
  # define parameters
  verbose, epochs, batch_size = 1, 1, 54
  n_features, n_outputs = train_x.shape[2], train_y.shape[1]
  # reshape into subsequences [samples, timesteps, rows, cols, channels]
  train_x = train_x.reshape((train_x.shape[0], n_steps, 1, n_length, n_features))
  # reshape output into [samples, timesteps, features]
  train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))
  # define model
  model = Sequential()
  model.add(ConvLSTM2D(filters=20, kernel_size=(1,3), activation='relu',input_shape=(n_steps, 1, n_length, n_features)))
  model.add(Flatten())
  model.add(RepeatVector(n_outputs))
  model.add(LSTM(20, activation='relu', return_sequences=True))
  #model.add(LSTM(20, activation='relu', return_sequences=True))
  model.add(TimeDistributed(Dense(100, activation='relu')))
  model.add(TimeDistributed(Dense(1)))
  model.compile(loss='mse', optimizer='adam')
  # fit network
  model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose)
  return model

# make a forecast
def forecast(model, history, n_steps, n_length, n_input):
  # flatten data
  data = array(history)
  data = data.reshape((data.shape[0]*data.shape[1], data.shape[2]))
  # retrieve last observations for input data
  input_x = data[-n_input:, :]
  # reshape into [samples, timesteps, rows, cols, channels]
  input_x = input_x.reshape((1, n_steps, 1, n_length, 2))
  # forecast the next week
  yhat = model.predict(input_x, verbose=0)
  # we only want the vector forecast
  yhat = yhat[0]
  return yhat

# evaluate a single model
def evaluate_model(train, test, n_steps, n_length, n_input):
  # fit model
  model = build_model(train, n_steps, n_length, n_input)
  # history is a list of weekly data
  history = [x for x in train]
  # walk-forward validation over each week
  predictions = list()
  for i in range(len(test)):
    # predict the next day
    yhat_sequence = forecast(model, history, n_steps, n_length, n_input)
    # store the predictions
    predictions.append(yhat_sequence)
    # get real observation and add to history for predicting the next week
    history.append(test[i, :])
  # evaluate predictions days for each week
  predictions = array(predictions)
  score, scores = evaluate_forecasts(test[:, :, 0], predictions)
  return score, scores,predictions


def loo_cv (array_2,lenght):
  house = array_2[:lenght]
  array_2 = array_2[lenght:]
  array_3 = np.concatenate((array_2,house))
  print(array_3.shape)
  return array_3

import random
random.seed(42)

#random_houses = [random.randint(0,99) for i in range(0,10)]
scores_cv = []


for i in range(1):
  data = loo_cv(data,6570)
  # split into train and test
  train, test = split_dataset(data)
  # define the number of subsequences and the length of subsequences
  n_steps, n_length = 5, 18
  # define the total days to use as input
  n_input = n_length * n_steps
  score, scores,predictions = evaluate_model(train, test, n_steps, n_length, n_input)
  # summarize scores
  summarize_scores('lstm', score, scores)
  scores_cv.append(score)
  print(data[-1])

print(scores_cv)

(657000, 2)
12163/12163 [==============================] - 544s 44ms/step - loss: 7.5057
lstm: [0.322] 0.4, 0.4, 0.4, 0.3, 0.3, 0.2, 0.3, 0.4, 0.3, 0.2, 0.3, 0.2, 0.3, 0.4, 0.4, 0.4, 0.4, 0.4
[0. 0.]
[0.3223120800405403]


#CV randomly selecting 10 houses for testing, afterwards averaging scores (rmse)

In [ ]:
import random

In [ ]:
random.seed(42)
random_houses = [random.randint(0, 99) for p in range(0, 10)]
print(random_houses)

[57, 12, 140, 125, 114, 71, 52, 279, 44, 216, 16, 15, 47, 111, 119, 258, 13, 287, 101, 279, 214, 112, 229, 142, 3, 81, 216, 174, 142, 79, 110, 172, 52, 47, 194, 49, 183, 176, 135, 22, 235, 274, 63, 193, 40, 282, 150, 185, 295, 98, 35, 23, 116, 148, 40, 119, 51, 194, 142, 232, 186, 83, 189, 181, 107, 136, 36, 87, 273, 125, 83, 236, 194, 138, 285, 112, 166, 28, 117, 16, 161, 205, 137, 33, 108, 290, 161, 108, 255, 202, 234, 73, 135, 71, 126, 287, 275, 134, 219, 298]


In [ ]:
### these functions don't work. Needs to be fixed ###

def cross_val (array, samples_division):
  start_index = samples_division * 6570
  end_index = (samples_division+1)* 6570
  test_subset = array((start_index, end_index))
  array = np.delete(array,test_subset)
  array = np.concatenate((array,test_subset ))
  return array

def cross_val(house_array,nth_house,length_house):
  end_index = nth_house*length_house
  start_index = end_index - length_house
  house = house_array[start_index:end_index]
  house_array = np.delete(house_array,np.s_[start_index:end_index], axis=0)
  print(house_array)
  house_array = np.concatenate((house_array,house))
  return house_array


def cross_val (array_house, samples_division,house_length):
  start_index = samples_division * house_length
  print(start_index)
  end_index = (samples_division+1)* house_length
  print(end_index)
  test_subset = array_house[start_index:end_index]
  print(test_subset)
  print(array_house)
  array_house = np.delete(array_house,test_subset, axis=0)
  print(array_house)
  array_house = np.concatenate((array_house,test_subset ))
  return array_house

def cross_val (array_house, samples_division,house_length):
  start_index = samples_division * house_length
  end_index = (samples_division+1)* house_length
  test_subset = array_house[start_index:end_index]
  array_house = np.delete(array_house,test_subset)
  array_house = np.concatenate((array_house,test_subset ))
  return array

def cross_val(house_array,nth_house,length_house):
  end_index = nth_house*length_house
  print(end_index)
  start_index = end_index - length_house
  print(start_index)
  house = house_array[start_index:end_index]
  print(house.shape)
  print(house)
  print(house_array.shape)
#  return delete
  house_array_2 = np.delete(house_array,np.s_[start_index:end_index], axis=0)
  print(house_array_2.shape)
  house_array_3 = np.concatenate((house_array_2,house))
  print(house_array_3.shape)
  return house_array_3

for i in l: 
  m = cross_val(m,i,2)
  print(m)

#cross_val(data,5,6570)
cross_val(a,5,2)
cross_val(a,0,2)  
cross_val(data,10,6570)

In [ ]:
scores_cv = []
for i in range(random_houses):
  data = cross_val(data,6570)
  # split into train and test
  train, test = split_dataset(data)
  # define the number of subsequences and the length of subsequences
  n_steps, n_length = 5, 18
  # define the total days to use as input
  n_input = n_length * n_steps
  score, scores,predictions = evaluate_model(train, test, n_steps, n_length, n_input)
  # summarize scores
  summarize_scores('lstm', score, scores)
  scores_cv.append(score)
  print(data[0])